# Surface Simplification Using Quadric Error Metrics

Implementation of the surface simplification algorithm described in the paper [Surface Simplification Using Quadric Error Metrics](https://www.cs.cmu.edu/~./garland/Papers/quadrics.pdf) by Michael Garland and Paul S. Heckbert.

### Algorithm Summary
1. Compute the quadric error matrix, $Q$, for each vertex.
2. Select all valid pairs.
3. Compute the optimal target vertex, $\bar{v}$ for each pair. $\bar{v}^T (Q_1 + Q_2) \bar{v}$ is the cost of this pair.
4. Find the pair with the lowest cost and collapse it.  This can be done with a heap to speed up the process.

### Setup
Enter the project root directory and run

`python3 -m venv venv`

`source venv/bin/activate`

`pip3 install -r requirements.txt`

Then run `jupyter notebook` and open `surface_simplification.ipynb`
or use vscode and select venv as the python interpreter.

In [1]:
import numpy as np
import plotly.graph_objects as go

### Load Starting Mesh

In [38]:
def parse_obj_file(obj_file):
    """
    Parses a .obj file and returns a list of vertices and a face_idicies
    :param obj_file: .obj file to parse
    :return: array of vertices, array of faces
    """
    vertices = []
    faces = []
    with open(obj_file, "r") as f:
        data = f.readlines()
        for line in data:
            tokens = line.split()
            if len(tokens) > 0:
                if tokens[0] == "v":
                    vertex = []
                    vertex.append(float(tokens[1]))
                    vertex.append(float(tokens[2]))
                    vertex.append(float(tokens[3]))
                    vertices.append(vertex)
                elif tokens[0] == "f":
                    vertex_idxs = []
                    for token_idx in range(1, 4):
                        vertex_idx = int(tokens[token_idx].split("/")[0])
                        vertex_idxs.append(vertex_idx)
                    faces.append(vertex_idxs)
                else:
                    continue

    vertices = np.array(vertices)
    faces = np.array(faces)
    faces = faces - 1  # Convert from 1 index to 0 index
    return vertices, faces


def visualize_mesh(vertices, faces):
    """
    Visualizes a mesh using plotly
    :param vertices: array of vertices
    :param faces: array of faces
    """
    mesh = go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        color="lightpink",
        opacity=0.50,
    )

    scatter = go.Scatter3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        mode="markers",
        marker=dict(size=2, color="blue"),
    )
    camera = dict(
        up=dict(x=0, y=1, z=0),
        center=dict(x=0, y=0, z=0),
    )

    fig = go.Figure(data=[mesh, scatter])
    fig.update_layout(scene_camera=camera, title="Original Plot")
    fig.show()

In [39]:
fname = "../assets/bunny_1k.obj"
vertices, faces = parse_obj_file(fname)
visualize_mesh(vertices, faces)

In [2]:
# Download data set from plotly repo
pts = np.loadtxt(
    np.DataSource().open(
        "https://raw.githubusercontent.com/plotly/datasets/master/mesh_dataset.txt"
    )
)
x, y, z = pts.T

fig = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z, color="lightpink", opacity=0.50)])
fig.show()